# 1.패키지 가져오기

In [ ]:
# 한글 불러옴
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

!apt-get update -qq
!apt-get install fonts-nanum* -qq

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 45 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Fetched 10.3 MB in 1s (9,891 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 121925 files and dire

In [ ]:
# 런타임 다시 시작

import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
from google.colab import drive
from scipy.cluster.hierarchy import linkage, fcluster, dendrogram
from sklearn.cluster import KMeans

# 나눔고딕 폰트 설정
font_path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
font_prop = fm.FontProperties(fname=font_path, size=12)
plt.rc('font', family='NanumGothic')

# 2.데이터 가져오기

In [ ]:
# CSV 파일 읽어오기 (예시로 각 파일명을 data1.csv, data2.csv, ...로 가정)
drive.mount('/content/drive')
df_cctv = pd.read_csv('/content/drive/MyDrive/[통계청 공모전] 음주운전/1 데이터/전처리 데이터/행정동별 cctv 개수.csv')
df_streetlamp = pd.read_csv('/content/drive/MyDrive/[통계청 공모전] 음주운전/1 데이터/전처리 데이터/행정동별 가로등 개수.csv')
df_busstop = pd.read_csv('/content/drive/MyDrive/[통계청 공모전] 음주운전/1 데이터/전처리 데이터/행정동별 버스 정류장 개수.csv')
df_restaurant = pd.read_csv('/content/drive/MyDrive/[통계청 공모전] 음주운전/1 데이터/전처리 데이터/행정동별 음식점 개수.csv')
df_car = pd.read_csv('/content/drive/MyDrive/[통계청 공모전] 음주운전/1 데이터/전처리 데이터/행정동별 자동차 소유 대수.csv')
df_bar = pd.read_csv('/content/drive/MyDrive/[통계청 공모전] 음주운전/1 데이터/전처리 데이터/행정동별 주점 개수.csv')
df_parking = pd.read_csv('/content/drive/MyDrive/[통계청 공모전] 음주운전/1 데이터/전처리 데이터/행정동별 주차장 개수.csv')
df_subway = pd.read_csv('/content/drive/MyDrive/[통계청 공모전] 음주운전/1 데이터/전처리 데이터/행정동별 지하철역 개수.csv')

Mounted at /content/drive


In [ ]:
dfs = [df_cctv, df_streetlamp, df_busstop, df_restaurant, df_car, df_bar, df_parking, df_subway]

In [ ]:
# 각 데이터프레임의 열 이름과 첫 번째 행의 값을 출력하는 함수
def print_column_names_and_first_row(dfs):
    for i, df in enumerate(dfs, 1):
        print(f"DataFrame {i}:")
        print("Columns:", df.columns.tolist())
        print("First Row:", df.iloc[0].tolist())
        print()

# 함수 호출
print_column_names_and_first_row(dfs)

# 1 열 이름 띄어쓰기
# 2 Unnamed: 0 열 존재

DataFrame 1:
Columns: ['Unnamed: 0', '행정동명', 'CCTV 개수', '행정동 코드']
First Row: [0, '강남구 개포1동', 57, 1168066000]

DataFrame 2:
Columns: ['Unnamed: 0', '행정동명', '가로등 개수', '행정동 코드']
First Row: [0, '강남구 개포3동', 102, 1168067500]

DataFrame 3:
Columns: ['Unnamed: 0', '행정동명', '버스 정류소 개수', '행정동 코드']
First Row: [0, '강남구 개포1동', 15, 1168066000]

DataFrame 4:
Columns: ['Unnamed: 0', '시군구명', '행정동명', '음식점 개수', '행정동코드']
First Row: [0, '강남구', '개포1동', 6, 1168066000]

DataFrame 5:
Columns: ['Unnamed: 0', '읍면동 (행정동)', '합계', '구', '동', '행정동코드']
First Row: [0, '강남구 개포1동 ', 3875, '강남구', '개포1동', 1168066000.0]

DataFrame 6:
Columns: ['Unnamed: 0', '시군구명', '행정동명', '주점 개수', '행정동코드']
First Row: [0, '강남구', '개포2동', 8, 1168067000]

DataFrame 7:
Columns: ['Unnamed: 0', '구', '동', '주차장 개수', '행정동 코드']
First Row: [0, '종로구', '사직동', 362, 1111053000]

DataFrame 8:
Columns: ['Unnamed: 0', '행정동명', '지하철역 개수', '행정동 코드']
First Row: [0, '강남구 개포2동', 3, 1168067000]



#3.데이터 통일시키기

In [ ]:
# 1,2 해결 함수
def preprocess(df):
    df = df.drop(columns=['Unnamed: 0'])  # 'Unnamed: 0' 열 제거
    df.columns = df.columns.str.replace(' ', '')  # 열 이름의 공백 제거
    return df

In [ ]:
# 각 데이터프레임 전처리
df_cctv = preprocess(df_cctv)
df_streetlamp = preprocess(df_streetlamp)
df_busstop = preprocess(df_busstop)
df_restaurant = preprocess(df_restaurant)
df_car = preprocess(df_car)
df_bar = preprocess(df_bar)
df_parking = preprocess(df_parking)
df_subway = preprocess(df_subway)

In [ ]:
dfs = [df_cctv, df_streetlamp, df_busstop, df_restaurant, df_car, df_bar, df_parking, df_subway]
# 함수 호출
print_column_names_and_first_row(dfs)

DataFrame 1:
Columns: ['행정동명', 'CCTV개수', '행정동코드']
First Row: ['강남구 개포1동', 57, 1168066000]

DataFrame 2:
Columns: ['행정동명', '가로등개수', '행정동코드']
First Row: ['강남구 개포3동', 102, 1168067500]

DataFrame 3:
Columns: ['행정동명', '버스정류소개수', '행정동코드']
First Row: ['강남구 개포1동', 15, 1168066000]

DataFrame 4:
Columns: ['시군구명', '행정동명', '음식점개수', '행정동코드']
First Row: ['강남구', '개포1동', 6, 1168066000]

DataFrame 5:
Columns: ['읍면동(행정동)', '합계', '구', '동', '행정동코드']
First Row: ['강남구 개포1동 ', 3875, '강남구', '개포1동', 1168066000.0]

DataFrame 6:
Columns: ['시군구명', '행정동명', '주점개수', '행정동코드']
First Row: ['강남구', '개포2동', 8, 1168067000]

DataFrame 7:
Columns: ['구', '동', '주차장개수', '행정동코드']
First Row: ['종로구', '사직동', 362, 1111053000]

DataFrame 8:
Columns: ['행정동명', '지하철역개수', '행정동코드']
First Row: ['강남구 개포2동', 3, 1168067000]



# 4. 데이터 병합하기

In [ ]:
# 각 데이터프레임의 기본 정보 출력 함수
def print_info(df, name):
    print(f"\n{name} 데이터프레임 정보:")
    print(f"Shape: {df.shape}")
    print(f"Null 값 개수:\n{df.isnull().sum()}")

In [ ]:
# 1. CCTV와 가로등 데이터 병합
print_info(df_cctv, "CCTV")
print_info(df_streetlamp, "가로등")
merged_df = df_cctv.merge(df_streetlamp, on=['행정동명','행정동코드'], how='outer')
print_info(merged_df, "CCTV와 가로등 병합")


CCTV 데이터프레임 정보:
Shape: (426, 3)
Null 값 개수:
행정동명      0
CCTV개수    0
행정동코드     0
dtype: int64

가로등 데이터프레임 정보:
Shape: (212, 3)
Null 값 개수:
행정동명     0
가로등개수    0
행정동코드    0
dtype: int64

CCTV와 가로등 병합 데이터프레임 정보:
Shape: (436, 4)
Null 값 개수:
행정동명        0
CCTV개수     10
행정동코드       0
가로등개수     224
dtype: int64


In [ ]:
# 2. 위 데이터와 버스 정류장 데이터 병합
print_info(df_busstop, "버스 정류장")
merged_df = merged_df.merge(df_busstop, on=['행정동명','행정동코드'], how='outer')
print_info(merged_df, "CCTV, 가로등과 버스 정류장 병합")


버스 정류장 데이터프레임 정보:
Shape: (426, 3)
Null 값 개수:
행정동명       0
버스정류소개수    0
행정동코드      0
dtype: int64

CCTV, 가로등과 버스 정류장 병합 데이터프레임 정보:
Shape: (436, 5)
Null 값 개수:
행정동명         0
CCTV개수      10
행정동코드        0
가로등개수      224
버스정류소개수     10
dtype: int64


In [ ]:
# 3. 위 데이터와 음식점 데이터 병합
print_info(df_restaurant, "음식점")
merged_df = merged_df.merge(df_restaurant, on=['행정동코드'], how='outer')
print_info(merged_df, "CCTV, 가로등, 버스 정류장과 음식점 병합")


음식점 데이터프레임 정보:
Shape: (425, 4)
Null 값 개수:
시군구명     0
행정동명     0
음식점개수    0
행정동코드    0
dtype: int64

CCTV, 가로등, 버스 정류장과 음식점 병합 데이터프레임 정보:
Shape: (436, 8)
Null 값 개수:
행정동명_x       0
CCTV개수      10
행정동코드        0
가로등개수      224
버스정류소개수     10
시군구명        11
행정동명_y      11
음식점개수       11
dtype: int64


In [ ]:
# 5. 위 데이터와 주점 데이터 병합
print_info(df_bar, "주점")
merged_df = merged_df.merge(df_bar, on=['행정동코드'], how='outer')
print_info(merged_df, "CCTV, 가로등, 버스 정류장, 음식점, 자동차 소유와 주점 병합")


주점 데이터프레임 정보:
Shape: (417, 4)
Null 값 개수:
시군구명     0
행정동명     0
주점개수     0
행정동코드    0
dtype: int64

CCTV, 가로등, 버스 정류장, 음식점, 자동차 소유와 주점 병합 데이터프레임 정보:
Shape: (436, 11)
Null 값 개수:
행정동명_x       0
CCTV개수      10
행정동코드        0
가로등개수      224
버스정류소개수     10
시군구명_x      11
행정동명_y      11
음식점개수       11
시군구명_y      19
행정동명        19
주점개수        19
dtype: int64


In [ ]:
# 6. 위 데이터와 주차장 데이터 병합
print_info(df_parking, "주차장")
merged_df = merged_df.merge(df_parking, on=['행정동코드'], how='outer')
print_info(merged_df, "CCTV, 가로등, 버스 정류장, 음식점, 자동차 소유, 주점과 주차장 병합")


주차장 데이터프레임 정보:
Shape: (426, 4)
Null 값 개수:
구        0
동        0
주차장개수    0
행정동코드    0
dtype: int64

CCTV, 가로등, 버스 정류장, 음식점, 자동차 소유, 주점과 주차장 병합 데이터프레임 정보:
Shape: (437, 14)
Null 값 개수:
행정동명_x       0
CCTV개수      10
행정동코드        0
가로등개수      224
버스정류소개수     10
시군구명_x      11
행정동명_y      11
음식점개수       11
시군구명_y      19
행정동명        19
주점개수        19
구           11
동           11
주차장개수       11
dtype: int64


## 병합 주의 필요

In [ ]:
dfs=[df_subway,df_car,merged_df]
print_column_names_and_first_row(dfs)

DataFrame 1:
Columns: ['행정동명', '지하철역개수', '행정동코드']
First Row: ['강남구 개포2동', 3, 1168067000]

DataFrame 2:
Columns: ['읍면동(행정동)', '합계', '구', '동', '행정동코드']
First Row: ['강남구 개포1동 ', 3875, '강남구', '개포1동', 1168066000.0]

DataFrame 3:
Columns: ['행정동명_x', 'CCTV개수', '행정동코드', '가로등개수', '버스정류소개수', '시군구명_x', '행정동명_y', '음식점개수', '시군구명_y', '행정동명', '주점개수', '구', '동', '주차장개수']
First Row: ['강남구 개포1동', 57.0, 1168066000, nan, 15.0, '강남구', '개포1동', 6.0, nan, nan, nan, '강남구', '개포1동', 30.0]



In [ ]:
# df_subway과 merged_df를 행정동명으로 병합
merged_df1_3 = merged_df.merge(df_subway, left_on='행정동명_x', right_on='행정동명', how='outer', suffixes=('_df3', '_df1'))

# 결과 출력
print("df1과 df3 병합 결과:")
merged_df1_3.head()

dfs=[merged_df1_3,df_car]
print_column_names_and_first_row(dfs)

df1과 df3 병합 결과:
DataFrame 1:
Columns: ['행정동명_x', 'CCTV개수', '행정동코드_df3', '가로등개수', '버스정류소개수', '시군구명_x', '행정동명_y', '음식점개수', '시군구명_y', '행정동명_df3', '주점개수', '구', '동', '주차장개수', '행정동명_df1', '지하철역개수', '행정동코드_df1']
First Row: ['강남구 개포1동', 57.0, 1168066000, nan, 15.0, '강남구', '개포1동', 6.0, nan, nan, nan, '강남구', '개포1동', 30.0, nan, nan, nan]

DataFrame 2:
Columns: ['읍면동(행정동)', '합계', '구', '동', '행정동코드']
First Row: ['강남구 개포1동 ', 3875, '강남구', '개포1동', 1168066000.0]



In [ ]:
merged_df1_3.head()

,행정동명_x,CCTV개수,행정동코드_df3,가로등개수,버스정류소개수,시군구명_x,행정동명_y,음식점개수,시군구명_y,행정동명_df3,주점개수,구,동,주차장개수,행정동명_df1,지하철역개수,행정동코드_df1
0,강남구 개포1동,57.0,1168066000,NaN,15.0,강남구,개포1동,6.0,NaN,NaN,NaN,강남구,개포1동,30.0,NaN,NaN,NaN
1,강남구 개포2동,231.0,1168067000,NaN,34.0,강남구,개포2동,94.0,강남구,개포2동,8.0,강남구,개포2동,154.0,강남구 개포2동,3.0,1.168067e+09
2,강남구 개포3동,108.0,1168067500,102.0,11.0,강남구,개포3동,43.0,강남구,개포3동,1.0,강남구,개포3동,29.0,강남구 개포3동,1.0,1.168068e+09
3,강남구 개포4동,381.0,1168069000,NaN,28.0,강남구,개포4동,141.0,강남구,개포4동,19.0,강남구,개포4동,1026.0,NaN,NaN,NaN
4,강남구 논현1동,478.0,1168052100,NaN,17.0,강남구,논현1동,548.0,강남구,논현1동,106.0,강남구,논현1동,2136.0,강남구 논현1동,2.0,1.168052e+09


In [ ]:
df_car.head()

,읍면동(행정동),합계,구,동,행정동코드
0,강남구 개포1동,3875,강남구,개포1동,1.168066e+09
1,강남구 개포2동,10266,강남구,개포2동,1.168067e+09
2,강남구 개포4동,8552,강남구,개포4동,1.168069e+09
3,강남구 논현1동,10388,강남구,논현1동,1.168052e+09
4,강남구 논현2동,10196,강남구,논현2동,1.168053e+09


### 홍제 1동, 제기동 문제

In [ ]:
# CSV 파일로 저장
merged_df1_3.to_csv('/content/drive/MyDrive/[통계청 공모전] 음주운전/1 데이터/전처리 데이터/merged_df1_3.csv', index=False, encoding='utf-8-sig')
df_car.to_csv('/content/drive/MyDrive/[통계청 공모전] 음주운전/1 데이터/전처리 데이터/df_car.csv', index=False, encoding='utf-8-sig')

In [ ]:
# 엑셀에서 전처리함.

In [ ]:
df_merge1 = pd.read_csv('/content/drive/MyDrive/[통계청 공모전] 음주운전/1 데이터/전처리 데이터/new_merge1 (1).csv')
df_car = pd.read_csv('/content/drive/MyDrive/[통계청 공모전] 음주운전/1 데이터/전처리 데이터/df_car.csv')

In [ ]:
df_code = pd.read_csv('/content/drive/MyDrive/[통계청 공모전] 음주운전/1 데이터/소현_유동인구 상권 및 시설 위치/행정동및법정동코드.csv')
df_code.to_csv('/content/drive/MyDrive/[통계청 공모전] 음주운전/1 데이터/전처리 데이터/행정동법정동한글버전.csv', index=False, encoding='utf-8-sig')

In [ ]:
# 엑셀에서 전처리함.

### 다시 병합

In [ ]:
df_merge1.head()

,행정동명_x,CCTV개수,행정동코드_df3,가로등개수,버스정류소개수,시군구명_x,행정동명_y,음식점개수,시군구명_y,행정동명_df3,주점개수,구,동,주차장개수,행정동명_df1,지하철역개수,행정동코드_df1
0,강남구 개포1동,57.0,1168066000,NaN,15.0,강남구,개포1동,6.0,NaN,NaN,NaN,강남구,개포1동,30.0,NaN,NaN,NaN
1,강남구 개포2동,231.0,1168067000,NaN,34.0,강남구,개포2동,94.0,강남구,개포2동,8.0,강남구,개포2동,154.0,강남구 개포2동,3.0,1.168067e+09
2,강남구 개포3동,108.0,1168067500,102.0,11.0,강남구,개포3동,43.0,강남구,개포3동,1.0,강남구,개포3동,29.0,강남구 개포3동,1.0,1.168068e+09
3,강남구 개포4동,381.0,1168069000,NaN,28.0,강남구,개포4동,141.0,강남구,개포4동,19.0,강남구,개포4동,1026.0,NaN,NaN,NaN
4,강남구 논현1동,478.0,1168052100,NaN,17.0,강남구,논현1동,548.0,강남구,논현1동,106.0,강남구,논현1동,2136.0,강남구 논현1동,2.0,1.168052e+09


In [ ]:
df_merge1.describe(include='all')

,행정동명_x,CCTV개수,행정동코드_df3,가로등개수,버스정류소개수,시군구명_x,행정동명_y,음식점개수,시군구명_y,행정동명_df3,주점개수,구,동,주차장개수,행정동명_df1,지하철역개수,행정동코드_df1
count,437,427.000000,4.370000e+02,213.000000,427.000000,426,426,426.000000,418,418,418.000000,426,426,426.000000,204,204.000000,2.040000e+02
unique,436,NaN,NaN,NaN,NaN,25,424,NaN,25,416,NaN,25,425,NaN,204,NaN,NaN
top,관악구 신사동,NaN,NaN,NaN,NaN,송파구,신사동,NaN,송파구,신사동,NaN,송파구,신사동,NaN,강남구 개포2동,NaN,NaN
freq,2,NaN,NaN,NaN,NaN,27,3,NaN,24,3,NaN,27,2,NaN,1,NaN,NaN
mean,NaN,130.489461,1.211715e+09,90.577465,26.433255,NaN,NaN,196.629108,NaN,NaN,40.349282,NaN,NaN,718.082160,NaN,1.328431,1.143745e+09
std,NaN,97.690042,4.469452e+08,109.826707,14.948648,NaN,NaN,183.087118,NaN,NaN,47.804784,NaN,NaN,535.294734,NaN,0.582990,1.998761e+07
min,NaN,1.000000,1.111052e+09,1.000000,1.000000,NaN,NaN,6.000000,NaN,NaN,1.000000,NaN,NaN,15.000000,NaN,1.000000,1.111053e+09
25%,NaN,51.000000,1.126069e+09,14.000000,15.000000,NaN,NaN,92.000000,NaN,NaN,13.000000,NaN,NaN,301.000000,NaN,1.000000,1.126059e+09
50%,NaN,112.000000,1.144071e+09,46.000000,24.000000,NaN,NaN,147.000000,NaN,NaN,26.000000,NaN,NaN,619.000000,NaN,1.000000,1.144060e+09
75%,NaN,193.500000,1.162062e+09,116.000000,34.000000,NaN,NaN,248.000000,NaN,NaN,52.000000,NaN,NaN,1066.000000,NaN,2.000000,1.165051e+09


In [ ]:
# '동' 값이 '신사동'인 행 필터링
sin_sa_dong_rows = df_merge1[df_merge1['동'] == '신사동']

# 결과 출력
print("동 값이 '신사동'인 행:")
print(sin_sa_dong_rows)


동 값이 '신사동'인 행:
     행정동명_x  CCTV개수   행정동코드_df3  가로등개수  버스정류소개수 시군구명_x 행정동명_y  음식점개수 시군구명_y  \
86  관악구 신사동   228.0  1162068500    2.0     17.0    관악구    신사동  147.0    관악구   
87  관악구 신사동   228.0  1162068500    2.0     17.0    관악구    신사동  147.0    관악구   

   행정동명_df3  주점개수    구    동   주차장개수 행정동명_df1  지하철역개수  행정동코드_df1  
86      신사동  41.0  관악구  신사동  2055.0      NaN     NaN        NaN  
87      신사동  41.0  강남구  신사동  1178.0      NaN     NaN        NaN  


In [ ]:
df_merge1.iloc[87]

행정동명_x          관악구 신사동
CCTV개수            228.0
행정동코드_df3    1162068500
가로등개수               2.0
버스정류소개수            17.0
시군구명_x              관악구
행정동명_y              신사동
음식점개수             147.0
시군구명_y              관악구
행정동명_df3            신사동
주점개수               41.0
구                   강남구
동                   신사동
주차장개수            1178.0
행정동명_df1            NaN
지하철역개수              NaN
행정동코드_df1           NaN
Name: 87, dtype: object

In [ ]:
# 87번째 행을 삭제 (인덱스가 0부터 시작하므로 86번 인덱스가 87번째 행에 해당)
df_merge1 = df_merge1.drop(df_merge1.index[87])

# 결과 출력
df_merge1.iloc[87]

행정동명_x          관악구 신원동
CCTV개수            193.0
행정동코드_df3    1162065500
가로등개수               NaN
버스정류소개수            14.0
시군구명_x              관악구
행정동명_y              신원동
음식점개수             125.0
시군구명_y              관악구
행정동명_df3            신원동
주점개수               21.0
구                   관악구
동                   신원동
주차장개수            1152.0
행정동명_df1            NaN
지하철역개수              NaN
행정동코드_df1           NaN
Name: 88, dtype: object

In [ ]:
df_car.describe(include='all')

,읍면동(행정동),합계,구,동,행정동코드
count,427,427.000000,427,427,4.270000e+02
unique,427,NaN,25,425,NaN
top,강남구 개포1동,NaN,송파구,신사동,NaN
freq,1,NaN,27,2,NaN
mean,NaN,7442.901639,NaN,NaN,1.143486e+09
std,NaN,3713.604729,NaN,NaN,1.924241e+07
min,NaN,545.000000,NaN,NaN,1.111052e+09
25%,NaN,4940.000000,NaN,NaN,1.126066e+09
50%,NaN,6879.000000,NaN,NaN,1.144066e+09
75%,NaN,8968.500000,NaN,NaN,1.162054e+09


In [ ]:
# '동' 값이 '신사동'인 행 필터링
sin_sa_dong_rows = df_car[df_car['동'] == '신사동']

# 결과 출력
print("동 값이 '신사동'인 행:")
print(sin_sa_dong_rows)


동 값이 '신사동'인 행:
   읍면동(행정동)    합계    구    동       행정동코드
14  강남구 신사동  7748  강남구  신사동  1168051000
87  관악구 신사동  5117  관악구  신사동  1162068500


In [ ]:
# 외부 조인을 수행하여 df_merge1와 df_car 병합
merged_df = df_merge1.merge(df_car, left_on='동', right_on='동', how='outer', suffixes=('_df1', '_df2'))

# 결과 출력
print("df_merge1와 df_car의 외부 조인 결과:")
print(merged_df)


df_merge1와 df_car의 외부 조인 결과:
       행정동명_x  CCTV개수   행정동코드_df3  가로등개수  버스정류소개수 시군구명_x 행정동명_y  음식점개수 시군구명_y  \
0    강남구 개포1동    57.0  1168066000    NaN     15.0    강남구   개포1동    6.0    NaN   
1    강남구 개포2동   231.0  1168067000    NaN     34.0    강남구   개포2동   94.0    강남구   
2    강남구 개포3동   108.0  1168067500  102.0     11.0    강남구   개포3동   43.0    강남구   
3    강남구 개포4동   381.0  1168069000    NaN     28.0    강남구   개포4동  141.0    강남구   
4    강남구 논현1동   478.0  1168052100    NaN     17.0    강남구   논현1동  548.0    강남구   
..        ...     ...         ...    ...      ...    ...    ...    ...    ...   
433  중랑구 상봉2동   189.0  1126059000   24.0     14.0    중랑구   상봉2동  334.0    중랑구   
434  중랑구 신내1동   172.0  1126068000  225.0     52.0    중랑구   신내1동  176.0    중랑구   
435  중랑구 신내2동    49.0  1126069000    8.0     20.0    중랑구   신내2동   77.0    중랑구   
436  중랑구 중화1동   159.0  1126060000    3.0     14.0    중랑구   중화1동   53.0    중랑구   
437  중랑구 중화2동   255.0  1126061000   89.0     28.0    중랑구   중화2동  230.0    중랑구   

In [ ]:
merged_df.columns

Index(['행정동명_x', 'CCTV개수', '행정동코드_df3', '가로등개수', '버스정류소개수', '시군구명_x', '행정동명_y',
       '음식점개수', '시군구명_y', '행정동명_df3', '주점개수', '구_df1', '동', '주차장개수',
       '행정동명_df1', '지하철역개수', '행정동코드_df1', '읍면동(행정동)', '합계', '구_df2', '행정동코드'],
      dtype='object')

In [ ]:
merged_df = merged_df[['행정동명_x', '시군구명_x', '행정동명_y', '행정동코드_df3','CCTV개수',  '가로등개수', '버스정류소개수','음식점개수', '주점개수', '주차장개수', '지하철역개수', '합계']]
merged_df.head()

,행정동명_x,시군구명_x,행정동명_y,행정동코드_df3,CCTV개수,가로등개수,버스정류소개수,음식점개수,주점개수,주차장개수,지하철역개수,합계
0,강남구 개포1동,강남구,개포1동,1168066000,57.0,NaN,15.0,6.0,NaN,30.0,NaN,3875.0
1,강남구 개포2동,강남구,개포2동,1168067000,231.0,NaN,34.0,94.0,8.0,154.0,3.0,10266.0
2,강남구 개포3동,강남구,개포3동,1168067500,108.0,102.0,11.0,43.0,1.0,29.0,1.0,6205.0
3,강남구 개포4동,강남구,개포4동,1168069000,381.0,NaN,28.0,141.0,19.0,1026.0,NaN,8552.0
4,강남구 논현1동,강남구,논현1동,1168052100,478.0,NaN,17.0,548.0,106.0,2136.0,2.0,10388.0


In [ ]:
merged_df.columns = ['행정동명', '시군구명','행정동명','행정동코드', 'CCTV개수',  '가로등개수', '버스정류소개수', '음식점개수', '주점개수', '주차장개수', '지하철역개수', '자동차소유대수']
merged_df.head()

,행정동명,시군구명,행정동명,행정동코드,CCTV개수,가로등개수,버스정류소개수,음식점개수,주점개수,주차장개수,지하철역개수,자동차소유대수
0,강남구 개포1동,강남구,개포1동,1168066000,57.0,NaN,15.0,6.0,NaN,30.0,NaN,3875.0
1,강남구 개포2동,강남구,개포2동,1168067000,231.0,NaN,34.0,94.0,8.0,154.0,3.0,10266.0
2,강남구 개포3동,강남구,개포3동,1168067500,108.0,102.0,11.0,43.0,1.0,29.0,1.0,6205.0
3,강남구 개포4동,강남구,개포4동,1168069000,381.0,NaN,28.0,141.0,19.0,1026.0,NaN,8552.0
4,강남구 논현1동,강남구,논현1동,1168052100,478.0,NaN,17.0,548.0,106.0,2136.0,2.0,10388.0


In [ ]:
merged_df.describe(include='all')

,행정동명,시군구명,행정동명,행정동코드,CCTV개수,가로등개수,버스정류소개수,음식점개수,주점개수,주차장개수,지하철역개수,자동차소유대수
count,438,427,427,4.380000e+02,428.000000,213.000000,428.000000,427.000000,419.000000,427.000000,205.000000,427.000000
unique,436,25,424,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,관악구 신사동,송파구,신사동,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,2,27,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,1.211629e+09,130.341121,90.577465,26.464953,196.470726,40.288783,719.627635,1.326829,7442.901639
std,NaN,NaN,NaN,4.464371e+08,97.623832,109.826707,14.945528,182.901384,47.763624,538.214798,0.582012,3713.604729
min,NaN,NaN,NaN,1.111052e+09,1.000000,1.000000,1.000000,6.000000,1.000000,15.000000,1.000000,545.000000
25%,NaN,NaN,NaN,1.126815e+09,51.000000,14.000000,15.000000,92.000000,13.000000,301.000000,1.000000,4940.000000
50%,NaN,NaN,NaN,1.144072e+09,112.000000,46.000000,24.000000,147.000000,26.000000,618.000000,1.000000,6879.000000
75%,NaN,NaN,NaN,1.162063e+09,193.250000,116.000000,34.000000,248.000000,52.000000,1064.000000,2.000000,8968.500000


In [ ]:
merged_df['행정동코드'] = merged_df['행정동코드'].astype(str)

merged_df = merged_df[merged_df['행정동코드'].str.startswith('11')]
merged_df.describe(include='all')

,행정동명,시군구명,행정동명,행정동코드,CCTV개수,가로등개수,버스정류소개수,음식점개수,주점개수,주차장개수,지하철역개수,자동차소유대수
count,428,427,427,428,428.000000,203.000000,428.000000,427.000000,419.000000,427.000000,205.000000,427.000000
unique,426,25,424,426,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,강동구 상일1동,송파구,신사동,1174052500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,2,27,3,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,130.341121,92.935961,26.464953,196.470726,40.288783,719.627635,1.326829,7442.901639
std,NaN,NaN,NaN,NaN,97.623832,110.859204,14.945528,182.901384,47.763624,538.214798,0.582012,3713.604729
min,NaN,NaN,NaN,NaN,1.000000,1.000000,1.000000,6.000000,1.000000,15.000000,1.000000,545.000000
25%,NaN,NaN,NaN,NaN,51.000000,16.000000,15.000000,92.000000,13.000000,301.000000,1.000000,4940.000000
50%,NaN,NaN,NaN,NaN,112.000000,49.000000,24.000000,147.000000,26.000000,618.000000,1.000000,6879.000000
75%,NaN,NaN,NaN,NaN,193.250000,122.500000,34.000000,248.000000,52.000000,1064.000000,2.000000,8968.500000


In [ ]:
merged_df = merged_df.fillna(0)
merged_df.isnull().sum()

행정동명       0
시군구명       0
행정동명       0
행정동코드      0
CCTV개수     0
가로등개수      0
버스정류소개수    0
음식점개수      0
주점개수       0
주차장개수      0
지하철역개수     0
자동차소유대수    0
dtype: int64

# 5. 최종 데이터 확인 및 저장

In [ ]:
# 최종 병합된 데이터프레임의 정보 출력
print_info(merged_df, "최종 병합된 데이터프레임")

# describe 함수 사용: 데이터프레임의 기초 통계량을 출력
print("\n기초 통계량:")
print(merged_df.describe(include='all'))

# head 함수 사용: 데이터프레임의 첫 몇 행을 출력
print("\n첫 5개 행:")
print(merged_df.head())


최종 병합된 데이터프레임 데이터프레임 정보:
Shape: (428, 12)
Null 값 개수:
행정동명       0
시군구명       0
행정동명       0
행정동코드      0
CCTV개수     0
가로등개수      0
버스정류소개수    0
음식점개수      0
주점개수       0
주차장개수      0
지하철역개수     0
자동차소유대수    0
dtype: int64

기초 통계량:
            행정동명 시군구명 행정동명       행정동코드      CCTV개수       가로등개수     버스정류소개수  \
count        428  428  428         428  428.000000  428.000000  428.000000   
unique       426   26  425         426         NaN         NaN         NaN   
top     강동구 상일1동  송파구  신사동  1174052500         NaN         NaN         NaN   
freq           2   27    3           2         NaN         NaN         NaN   
mean         NaN  NaN  NaN         NaN  130.341121   44.079439   26.464953   
std          NaN  NaN  NaN         NaN   97.623832   89.288881   14.945528   
min          NaN  NaN  NaN         NaN    1.000000    0.000000    1.000000   
25%          NaN  NaN  NaN         NaN   51.000000    0.000000   15.000000   
50%          NaN  NaN  NaN         NaN  112.000000    0.000000   2

In [ ]:
# 최종 병합된 데이터프레임을 CSV 파일로 저장 (UTF-8 인코딩)
output_file_path = '/content/drive/MyDrive/[통계청 공모전] 음주운전/1 데이터/전처리 데이터/final_merged_data.csv'  # 원하는 파일 경로 및 이름 지정

# CSV 파일로 저장
merged_df.to_csv(output_file_path, index=False, encoding='utf-8-sig')

print(f"최종 병합된 데이터프레임이 '{output_file_path}' 경로에 UTF-8 인코딩으로 저장되었습니다.")


최종 병합된 데이터프레임이 '/content/drive/MyDrive/[통계청 공모전] 음주운전/1 데이터/전처리 데이터/final_merged_data.csv' 경로에 UTF-8 인코딩으로 저장되었습니다.
